In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

In [3]:
!unzip -o /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/validation-20220402T110325Z-001.zip
! unzip -o /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/val_11_16_data.zip -d validation/
! unzip -o /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/val_11_16_labels.zip -d validation/

Archive:  /content/drive/MyDrive/CS4220_mp2/CS4220/datasets/validation-20220402T110325Z-001.zip
  inflating: validation/val7_labels.txt  
  inflating: validation/val8_labels.txt  
  inflating: validation/val10_labels.txt  
  inflating: validation/val9_labels.txt  
  inflating: validation/val6_labels.txt  
  inflating: validation/val3_labels.txt  
  inflating: validation/val5_abundance.txt  
  inflating: validation/val3_abundance.txt  
  inflating: validation/val4_labels.txt  
  inflating: validation/val1_labels.txt  
  inflating: validation/val9_abundance.txt  
  inflating: validation/val10_abundance.txt  
  inflating: validation/val6_abundance.txt  
  inflating: validation/val1_abundance.txt  
  inflating: validation/val5_labels.txt  
  inflating: validation/val7_abundance.txt  
  inflating: validation/val2_labels.txt  
  inflating: validation/val4_abundance.txt  
  inflating: validation/val2_abundance.txt  
  inflating: validation/val8_abundance.txt  
  inflating: validation/genome_r

In [4]:
!pip install umap-learn[plot]
!pip install fastparquet
import umap
import umap.plot

     |████████████████████████████████| 86 kB 2.0 MB/s 
     |████████████████████████████████| 1.1 MB 9.0 MB/s 
     |████████████████████████████████| 15.8 MB 34.4 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 136 kB 47.6 MB/s 
     |████████████████████████████████| 837 kB 46.1 MB/s 
     |████████████████████████████████| 830 kB 63.4 MB/s 
     |████████████████████████████████| 822 kB 44.2 MB/s 
     |████████████████████████████████| 802 kB 47.1 MB/s 
     |████████████████████████████████| 802 kB 60.6 MB/s 
     |████████████████████████████████| 793 kB 57.3 MB/s 
     |████████████████████████████████| 793 kB 55.3 MB/s 
     |████████████████████████████████| 791 kB 55.3 MB/s 
     |████████████████████████████████| 786 kB 44.1 MB/s 
     |████████████████████████████████| 779 kB 57.7 MB/s 
     |████████████████████████████████| 778 kB 61.5 MB/s 
     |████████████████████████████████| 776 kB 60.2 MB/s 
     |████████████

     |████████████████████████████████| 1.5 MB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 44.1 MB/s 


In [5]:
labels_df = pd.read_csv("./drive/MyDrive/CS4220_mp2/CS4220/datasets/labels.csv")
labels_df.head()

,Unnamed: 0,genome_name,rl,rq
0,0,staphylococcus_aureus,962,98.26
1,0,staphylococcus_aureus,962,98.26
2,1,staphylococcus_aureus,704,90.25
3,2,staphylococcus_aureus,2617,96.20
4,3,staphylococcus_aureus,4169,99.38


In [6]:
df_train = pd.read_csv("./drive/MyDrive/CS4220_mp2/CS4220/datasets/perfect_reads_6mer.gz", header=None, sep="\t", nrows=100000)
df_train["labels"] = labels_df["genome_name"][0:100000]
print(df_train.shape)
df_train.head()

(100000, 2082)


,0,1,2,3,4,5,6,7,8,9,...,2072,2073,2074,2075,2076,2077,2078,2079,2080,labels
0,0.003247,0.002165,0.002165,0.000000,0.004329,0.002165,0.001082,0.000000,0.001082,0.001082,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,staphylococcus_aureus
1,0.006237,0.004158,0.001040,0.002079,0.003119,0.003119,0.004158,0.001040,0.004158,0.000000,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,staphylococcus_aureus
2,0.004261,0.002841,0.001420,0.004261,0.005682,0.007102,0.005682,0.002841,0.001420,0.002841,...,0.00142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,staphylococcus_aureus
3,0.000000,0.001528,0.002293,0.001146,0.001911,0.001911,0.000764,0.001528,0.003439,0.001911,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,staphylococcus_aureus
4,0.001919,0.003838,0.001679,0.002399,0.003838,0.002878,0.001919,0.002399,0.002399,0.001199,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,staphylococcus_aureus


In [7]:
df_train.drop(2080, inplace=True, axis = 1)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,2071,2072,2073,2074,2075,2076,2077,2078,2079,labels
0,0.003247,0.002165,0.002165,0.000000,0.004329,0.002165,0.001082,0.000000,0.001082,0.001082,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus
1,0.006237,0.004158,0.001040,0.002079,0.003119,0.003119,0.004158,0.001040,0.004158,0.000000,...,0.002079,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus
2,0.004261,0.002841,0.001420,0.004261,0.005682,0.007102,0.005682,0.002841,0.001420,0.002841,...,0.000000,0.00142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus
3,0.000000,0.001528,0.002293,0.001146,0.001911,0.001911,0.000764,0.001528,0.003439,0.001911,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus
4,0.001919,0.003838,0.001679,0.002399,0.003838,0.002878,0.001919,0.002399,0.002399,0.001199,...,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,staphylococcus_aureus


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train["labels_encoded"] = le.fit_transform(df_train["labels"])

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42, max_depth = 6)
rf.fit(df_train.iloc[:, 0:2080], df_train["labels_encoded"].values)

RandomForestClassifier(max_depth=6, random_state=42)

In [10]:
 import pickle
 pickle.dump( rf, open( "./drive/MyDrive/CS4220_mp2/rf_100000_cluster_and_classify.pkl", "wb" ) )

In [11]:
def precision_per_patient(patient_id, preds):
    df_true = pd.read_csv('./validation/val{}_labels.txt'.format(patient_id))
    tp, fp=0,0
    print('my predition(s) for patient {}:'.format(patient_id))
    print(preds)
    print('true pathogen')
    print(df_true['true_label'].values)
    for item in np.unique(preds):
        if item in df_true['true_label'].values:
            tp+=1
        else:
            fp+=1
    return tp/(tp+fp)

In [13]:
# fig, ax = plt.subplots(16, 1, figsize=(20,20))
all_precision = []
for patient_id in range(1,17):
    # kmeans = KMeans(random_state=42, n_clusters = 2)
    gmm = GaussianMixture(n_components=2, covariance_type="full")
    print('predicting for patient {}'.format(patient_id))
    df_test = pd.read_csv('./validation/val{}_6mer.gz'.format(patient_id), header=None, sep='\t')
    # mapped = umap.UMAP().fit_transform(df_test.iloc[:,:-1])
    df_test["clusters"] = gmm.fit_predict(df_test.iloc[:,:-1])
    # sns.scatterplot(x=mapped[:,0], y=mapped[:, 1], ax=ax[patient_id-1])
    # ax[patient_id] = umap.plot.points(mapped, width=100, height=100)
    min_cluster_idx = df_test["clusters"].value_counts().sort_values().index[0]
    subset = df_test[df_test["clusters"] == min_cluster_idx]
    ypred = rf.predict(subset.iloc[:,:-2])
    final_predictions = le.inverse_transform(np.unique(ypred))
    precision = precision_per_patient(patient_id, final_predictions)
    print('precision: {}'.format(precision))
    all_precision.append(precision)

predicting for patient 1
my predition(s) for patient 1:
['burkholderia_pseudomallei' 'decoy']
true pathogen
['staphylococcus_aureus']
precision: 0.0
predicting for patient 2
my predition(s) for patient 2:
['burkholderia_pseudomallei' 'decoy']
true pathogen
['staphylococcus_pyogenes']
precision: 0.0
predicting for patient 3
my predition(s) for patient 3:
['decoy']
true pathogen
['burkholderia_pseudomallei' 'corynebacterium_ulcerans']
precision: 0.0
predicting for patient 4
my predition(s) for patient 4:
['decoy' 'pseudomonas_aeruginosa']
true pathogen
['pseudomonas_aeruginosa']
precision: 0.5
predicting for patient 5
my predition(s) for patient 5:
['decoy']
true pathogen
['corynebacterium_diphtheriae']
precision: 0.0
predicting for patient 6
my predition(s) for patient 6:
['decoy']
true pathogen
['streptococcus_pneumoniae']
precision: 0.0
predicting for patient 7
my predition(s) for patient 7:
['burkholderia_pseudomallei' 'decoy' 'mycobacterium_ulcerans'
 'pseudomonas_aeruginosa']
true 

In [14]:
print([ 'patient {}: {}'.format(c,item) for c, item in enumerate(all_precision, start=1)], 'avg: {}'.format(np.mean(all_precision)))

['patient 1: 0.0', 'patient 2: 0.0', 'patient 3: 0.0', 'patient 4: 0.5', 'patient 5: 0.0', 'patient 6: 0.0', 'patient 7: 0.25', 'patient 8: 0.0', 'patient 9: 0.0', 'patient 10: 0.5', 'patient 11: 1.0', 'patient 12: 0.0', 'patient 13: 0.0', 'patient 14: 0.0', 'patient 15: 0.0', 'patient 16: 1.0'] avg: 0.203125
